## The Knapsack Problem & Genetic Algorithms - Computerphile
[Source](https://www.youtube.com/watch?v=MacVqujSXWE)

### Maximize profit without going over weight limit

In [6]:
# # https://www.dataminingapps.com/2017/03/solving-the-knapsack-problem-with-a-simple-genetic-algorithm/
# # Define data 
# p = [6, 5, 8, 9, 6, 7, 3] # Profits 
# w = [2, 3, 6, 7, 5, 9, 4]# Weights 
# W = 9 # Knapsack ’s capacity 
# n = len(p) # Number of items

In [7]:
# Define data 
p = [5, 4, 7, 2] # Profits 
w = [7, 2, 1, 9] # Weight 
W = 15 # Knapsack ’s capacity 
n = len(p) # Number of items

In [91]:
import random
import pandas as pd

In [44]:
def getWeightOfChoices(seq):
    allweights = [a * b for a, b in zip(seq, w)]
    return allweights, sum(allweights)

def getProfitOfChoices(seq):
    allprofits = [a * b for a, b in zip(seq, p)]
    return allprofits,sum(allprofits)

In [70]:
ranPick = lambda: random.sample([0,1],1)[0]
def makePopulation(popsize=2,nBits=n):
    return [[ranPick() for _ in range(nBits)] for p in range(popsize)]

In [83]:
def getFitness(comb,maxW=W):
    _,totalprofit =  getProfitOfChoices(comb)
    _,totalweight =  getWeightOfChoices(comb)
    if totalweight>maxW:
        totalprofit = 0
    return totalprofit,totalweight

In [247]:
popsize = 8
population = makePopulation(popsize)

### Get fitness of population

In [284]:
def fitnessOfPopulation(population):
    fitnessXpopulation = [getFitness(ind) for ind in population]
    fitnessDF = pd.DataFrame(fitnessXpopulation,columns=['score','weight'])
    fitnessDF['solution'] = population
    return fitnessDF, fitnessDF.score.mean(),fitnessDF.weight.mean()

### Selection (tournament)


In [238]:
def selectParents(population):
    '''
    Select 2 pairs of parents at random
    '''
    population_copy = population.copy()
    random.shuffle(population_copy)
    a = population_copy.pop()
    random.shuffle(population_copy)
    b = population_copy.pop()
    random.shuffle(population_copy)
    x = population_copy.pop()
    random.shuffle(population_copy)
    y = population_copy.pop()
    return a,b,x,y

def getWiningParents(a,b,x,y):
    a_score,b_score,x_score,y_score = [getFitness(_)[0] for _ in [a,b,x,y]]
    ###
    winnerA =  a if a_score > b_score else b
    winnerX =  x if x_score > y_score else y
    return winnerA,winnerX    

def recombine(winnerA,winnerX,point=int(n/2)):
    offspringA = winnerA[:point]+winnerX[point:]
    offspringB = winnerX[:point]+winnerA[point:]
    return offspringA,offspringB

tosscoin = lambda: random.uniform(0,1)
mutationrate = .05
def flipbit(bit):
    return 1 if bit==0 else 0

def mutationProcess(bit):
    if tosscoin() <= mutationrate:
        return flipbit(bit)
    else:
        return bit
    
mutateOffspring = lambda child: [mutationProcess(_) for _ in child]    

def mutateChildren(A,B):
    childANew = mutateOffspring(A)
    childBNew = mutateOffspring(B)
    return childANew,childBNew

In [273]:
def getOffspring(population):
    a,b,x,y = selectParents(population)
    winnerA,winnerX  = getWiningParents(a,b,x,y)
    offspringA,offspringB = recombine(winnerA,winnerX)
    childANew,childBNew = mutateChildren(offspringA,offspringB)
    return childANew,childBNew

In [274]:
def getNewGeneration(population):
    newgen = []
    while len(newgen) < popsize:
        childANew,childBNew = getOffspring(population)
        newgen.append([childANew,childBNew])
    newgen = sum(newgen,[])
    return newgen

8.375

In [276]:
## First generation
newpop = getNewGeneration(population)
popDF, avgFit = fitnessOfPopulation(newpop) 
avgFit

5.9375

In [278]:
## First generation
newpop = getNewGeneration(newpop)
popDF, avgFit = fitnessOfPopulation(newpop) 
avgFit

9.3125

In [301]:
popDF = dict()
avgWeight = dict()
avgFit = dict()
popDF[0], avgFit[0],avgWeight[0]=fitnessOfPopulation(population) 
newpop = getNewGeneration(newpop)
for gen in range(1,1000):
    newpop = getNewGeneration(newpop)
    popDF[gen], avgFit[gen],avgWeight[gen]=fitnessOfPopulation(newpop) 
    popDF[gen] = popDF[gen].sort_values('score',ascending=False).reset_index(drop=True)

In [302]:
avgFit[gen]

16.0

In [303]:
popDF[99]

,score,weight,solution
0,16,10,"[1, 1, 1, 0]"
1,16,10,"[1, 1, 1, 0]"
2,16,10,"[1, 1, 1, 0]"
3,16,10,"[1, 1, 1, 0]"
4,16,10,"[1, 1, 1, 0]"
5,16,10,"[1, 1, 1, 0]"
6,16,10,"[1, 1, 1, 0]"
7,16,10,"[1, 1, 1, 0]"
8,16,10,"[1, 1, 1, 0]"
9,16,10,"[1, 1, 1, 0]"
